In [ ]:
# Basic imports
import pandas as pd
import numpy as np

# Machine Learning libraries
from sklearn.model_selection import train_test_split

# Loading of the dataset via pandas
kc_data = pd.read_csv("data/King_County_House_prices_dataset.csv")

In [ ]:
### Data Cleaning

# We will drop this row
kc_data.drop(15856, axis=0, inplace=True)

# We are calculating the "sqft_basement" by substracting sqft_above of sqft_living
kc_data.eval('sqft_basement = sqft_living - sqft_above', inplace=True)



# Summation of the missing values and calculation of the missing values as a percentage
missing_values = pd.DataFrame(kc_data.isnull().sum(),columns=['count'])
missing_values['percentage'] = (missing_values['count']/kc_data.shape[0]*100).round(2)
missing_values.query('count != 0')



# We replace Nan values in "view" with the most frequent expression (0)
kc_data['view'].fillna(0, inplace=True)

# We display how often the different values of the variable occur.
kc_data.waterfront.value_counts()

# We replace Nan values in "waterfront" with the most frequent expression (0)
kc_data.waterfront.fillna(0, inplace=True)



# We will create an empty list in which we will store values
last_known_change = []

# For each row in our data frame, we look at what is in the column "yr_renovated".
for idx, yr_re in kc_data.yr_renovated.iteritems():
    # if "yr_renovated" is 0 or contains no value, we store the year of construction of the house in our empty listes ab
    if str(yr_re) == 'nan' or yr_re == 0.0:
        last_known_change.append(kc_data.yr_built[idx])
    # if there is a value other than 0 in the column "yr_renovated", we transfer this value into our new list
    else:
        last_known_change.append(int(yr_re))

# We create a new column and take over the values of our previously created list
kc_data['last_known_change'] = last_known_change

# We delete the "yr_renovated" and "yr_built" columns
kc_data.drop("yr_renovated", axis=1, inplace=True)
kc_data.drop("yr_built", axis=1, inplace=True)

In [ ]:
### Feature Engineering
### price per sqft
kc_data['sqft_price'] = (kc_data.price/(kc_data.sqft_living + kc_data.sqft_lot)).round(2)

In [ ]:
### Distance to the center of wealth
kc_data[kc_data['sqft_price']==kc_data['sqft_price'].max()]

kc_data[kc_data['price']==kc_data['price'].max()]

# Absolute difference of latitude between centre and property
kc_data['delta_lat'] = np.absolute(47.62774- kc_data['lat'])
# Absolute difference of longitude between centre and property
kc_data['delta_long'] = np.absolute(-122.24194-kc_data['long'])
# Distance between centre and property
kc_data['center_distance']= ((kc_data['delta_long']* np.cos(np.radians(47.6219)))**2 
                                   + kc_data['delta_lat']**2)**(1/2)*2*np.pi*6378/360

In [ ]:
# This function helps us to calculate the distance between the house overlooking the seafront and the other houses.
def dist(long, lat, ref_long, ref_lat):
    '''dist computes the distance in km to a reference location. Input: long and lat of 
    the location of interest and ref_long and ref_lat as the long and lat of the reference location'''
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

# All houses with "waterfront" are added to the list
water_list= kc_data.query('waterfront == 1')
water_list.head()

water_distance = []
# For each row in our data frame we now calculate the distance to the seafront
for idx, lat in kc_data.lat.iteritems():
    ref_list = []
    for x,y in zip(list(water_list.long), list(water_list.lat)):
        ref_list.append(dist(kc_data.long[idx], kc_data.lat[idx],x,y).min())
    water_distance.append(min(ref_list))

# wir erstellen eine neue Spalte und übernehmen die Werte unserer vorher erstellten Liste
kc_data['water_distance'] = water_distance



In [ ]:
### Modelling preparation
# We have to remove these columns because of Data Leakage: price or because they do not provide prognostic information.
drop_lst = ['price', 'sqft_price', 'date', 'delta_lat', 'delta_long',]

# we would like to consider all variables except the ones mentioned above
all_features = [x for x in kc_data.columns if x not in drop_lst]

# X contains all descriptive variables defined above
X = kc_data[all_features]

# we define y (our dependent variable): we take the price
y = kc_data.price

### Move this to the front!
# We separate our data into train and test data. In the process, 30 % of the data is used for the subsequent testing of the prognostic quality.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


